# Hugging Face Library 'Transformer'およびT5Tokenizerのダウンロード

参考(https://qiita.com/takubb/items/fd972f0ac3dba909c293)これを基に改造し、最新のGoogle Colaboratoryで動作するようにした

In [1]:
!export CUDA_LAUNCH_BLOCKING=1
# !pip install torch
!pip install torchvision
!pip install transformers
#!apt install swig
# Sentencepieceのインストール
!pip install sentencepiece
!pip install mecab-python3
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 
import gzip
import shutil
import sqlite3
import random
from math import ceil
import re
import csv
import glob
import torchvision
import statistics
import numpy as np
import lightgbm
import pickle
import statistics
import MeCab
import re
import copy

import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

# PyTorchとGPU設定

In [3]:
#!pip install torch
import torch
# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Data Augmentation kansuu

In [4]:
# synreplace - replace kasho kosuu
# randinsert - tasu kotoba no kazu
# randdelete - delete kakuritsu
# randswap - swap kaisuu

class synreplace(object):
    def __init__(self, num):
        self.num = num
        self.model = RobertaForMaskedLM.from_pretrained("rinna/japanese-roberta-base")
    def __call__(self, textlist):
        # textlist: honbun no list
        textlen = torch.where(textlist == 3)[0][0]
        for n in range(self.num):
            # chikan shiro
            masked_idx = random.randint(2, textlen-1)
            textlist[masked_idx] = 6
            # convert to tensor
            token_tensor = torch.tensor(textlist)
            # get the top 10 predictions of the masked token
            self.model = self.model.eval()
            with torch.no_grad():
                outputs = self.model(torch.unsqueeze(token_tensor, 0))
                predictions = outputs[0][0, masked_idx].topk(1)
            for i, index_t in enumerate(predictions.indices):
                index = index_t.item()
            textlist[masked_idx] = index
        return textlist

class randinsert(object):
    def __init__(self, num):
        self.num = num
    def __call__(self, textlist):
        for n in range(self.num):
            insword = textlist[random.randint(1,len(textlist)-1)]
            i = random.randint(1,len(textlist)-1)
#            print('len: ', len(textlist))
#            print(i)
            while textlist[i] == 3:
                i = random.randint(1,len(textlist)-1)
#                print(i)
            textlist = torch.cat([textlist[0:i], torch.tensor([insword]), textlist[i:-1]])
        return textlist

class randdelete(object):
    def __init__(self, num):
        self.num = num
    def __call__(self, textlist):
#        print(textlist.shape)
        for i in range(3,len(textlist)-1):
            if textlist[i] == 3:
                continue
            r = random.uniform(0, 1)
            if r < self.num:
#                textlist.pop(i)
                textlist = torch.cat([textlist[0:i], textlist[i+1:], torch.tensor([3])])
#                print(textlist)
        return textlist

class randswap(object):
    def __init__(self, num):
        self.num = num
    def __call__(self, textlist):
        counter = 0
        #rs_sents = np.zeros(len(textlist), dtype=object)
        for i in range(len(textlist)):
            while self.num > counter:
                box = 0
                random_idx_1 = random.randint(1, len(textlist)-1)
                while textlist[random_idx_1] == 3:
                    random_idx_1 = random.randint(0, len(textlist)-1)
                random_idx_2 = random.randint(1, len(textlist)-1)
                while random_idx_1 == random_idx_2 or textlist[random_idx_2] == 3:
                    random_idx_2 = random.randint(0, len(textlist)-1)
                    # print(random_idx_1, random_idx_2)
                box = textlist[random_idx_1]
                textlist[random_idx_1] = textlist[random_idx_2]
                textlist[random_idx_2] = box
                counter += 1
        return textlist

# Custom Tensor Dataset
https://stackoverflow.com/questions/55588201/pytorch-transforms-on-tensordataset

# データセットの準備

ライブドアニュースコーパスをダウンロード

    ダウンロードしたファイルは圧縮（tar.gz形式）ファイル
    様々なジャンル（IT,スポーツ,家電,映画など）のWEBメディアごとにフォルダに記事がテキストファイルで保存されている
    
以下、ファイルを読み込んで、必要な部分を抽出

In [5]:
#urllib.request.urlretrieve("https://www.rondhuit.com/download/ldcc-20140209.tar.gz", "ldcc-20140209.tar.gz")
# ダウンロードした圧縮ファイルのパスを設定
#tgz_fname = "ldcc-20140209.tar.gz" 
# 2つをニュースメディアのジャンルを選定
mydata = '/export/livedoor' 
#処理をした結果を保存するファイル名 
tsv_fname = "all_text.tsv" 

def remove_brackets(inp):
    output = re.sub(u'[〃-〿]', '',(re.sub('＝|=|×|\(|\)|“|”|（|）|／|\[|\]| |　|…|・|\n|\t|/|＜|＞|@|＠', '', re.sub(u'[ℊ-⿻]', '', inp)))) #210A ~ 2FFF
    return output

"""
def read_url(f):
    url = next(f)
    return url[:-1]

def read_date(f):
    date = next(f)
    date = remove_brackets(date.encode().decode('utf-8'))
    return date[:-1]
"""

def read_title(f):
    next(f)
    next(f)
    title = next(f)
    title = remove_brackets(title.encode().decode('utf-8'))
    return title[:-1]

def read_para(f):
    p = ''
    while True:
        try:
            para = next(f)
            para = remove_brackets(para.encode().decode('utf-8'))
            p += para
        except StopIteration:
            break
    return p [:-1]

In [6]:
directory = ['/export/livedoor/dokujo-tsushin', '/export/livedoor/it-life-hack']
target_genre = ["dokujo-tsushin", "it-life-hack"] 
#directory = ['/export/livedoor/dokujo-tsushin', '/export/livedoor/peachy']
#target_genre = ["dokujo-tsushin", "peachy"] 
zero_fnames = []
one_fnames = []

if os.path.exists(tsv_fname) == True:
    with open(tsv_fname, "r+") as f:
        f.truncate(0)

for i in range(2):
    for filename in os.listdir(directory[i]):
        if "LICENSE.txt" in filename:
            continue
        f = os.path.join(directory[i], filename)
        #if os.path.isfile(f):
        #    print(f)
        if target_genre[0] in f and f.endswith(".txt"):
            with open(tsv_fname, "a") as wf:
                writer = csv.writer(wf, delimiter='\t')
                with open(f) as zf:
                    title = read_title(zf)
                    para = read_para(zf)
                    row = [target_genre[0], '0', title, para]
                    writer.writerow(row)
            continue
        if target_genre[1] in f and f.endswith(".txt"):
            with open(tsv_fname, "a") as wf:
                writer = csv.writer(wf, delimiter='\t')
                with open(f) as zf:
                    title = read_title(zf)
                    para = read_para(zf)
                    row = [target_genre[1], '1', title, para]
                    writer.writerow(row)
            continue

pandasでデータを読み込み

In [7]:
import pandas as pd
# データの読み込み
df = pd.read_csv("all_text.tsv", 
                 delimiter='\t', header=None, names=['media_name', 'label','title','sentence'])
# データの確認
print(f'データサイズ： {df.shape}')
df.sample(10)

データサイズ： (1740, 4)


,media_name,label,title,sentence
936,it-life-hack,1,おはこんこん、ふぉっくす紺子だよ！新しいおしらせ紺子にゅう,おはこんこんふぉっくす紺子だよ今日は大きいにゅうすがふたつあるんだよ!!さいきんおなじにゅー...
322,dokujo-tsushin,0,恋愛相談で頼れるのはアノ友,仕事に趣味に忙しく、恋愛なんてすっかりご無沙汰。そんな独女だって、ある日、突然恋に落ちること...
651,dokujo-tsushin,0,独女が結婚で捨てられないも,今年8月で35歳になるチズルさん教育関係は、親戚から持ち込まれたお見合い話に心が揺れている。...
203,dokujo-tsushin,0,言うに言えない！突然Ｈを断ってしまった理,彼といい雰囲気になって、いよいよベッドへという時になってやっぱり今日は家に帰るわ、ごめんなさ...
523,dokujo-tsushin,0,あなたは大丈夫？ファッションの聖地でイモ女を大捜,9月20日、台風15号の接近によって真夏日から解放された代わりに、翌日には震災の悪夢が蘇る帰...
922,it-life-hack,1,3万円台のコンパクトベアボーンZOTACのベアボーンに新モデルが登,ZOTACZBOX-ID84-PLUSアスクは、ZOTACのベアボーンシステムZBOXシリー...
1528,it-life-hack,1,ゲリラ豪雨にも勝てる？iPhoneをマイ自転車に搭載できる強力な防水機能マウントイケショップのレア,今夏、都内は猛暑日が続く中、オリンピック観戦で、スポーツしたい！と思ってしまった人も多いかと...
341,dokujo-tsushin,0,婚期短し恋せよ乙女！乙女女子解体新,近年、大人女子、肉食女子、腐女子etcといったある種の女性たちのことを女子と総称する呼び名が...
467,dokujo-tsushin,0,女同士の話が噛み合わない理由はしゃべり方にある!,学生時代に仲が良かった友人同士でも、大人になり、普段の生活環境や立場が変わり、話が合わなくな...
504,dokujo-tsushin,0,恋愛感情がイマイチでも結婚はできるのか,婚活ブームの影響で、絶命寸前と言われていたお見合いが、再び脚光を浴びている。結婚相談所経由の...


//文章データをsentences、ラベルデータを labelsに保存、以降この2変数だけを利用

In [8]:
mn = df.media_name.values
labels = df.label.values
titles = df.title.values
sentences = df.sentence.values

In [9]:
tagger = MeCab.Tagger("-Owakati")

def make_wakati(sentence):
  # MeCabで分かち書きを行う
    sentence = tagger.parse(sentence)
  # 半角全角英数字などは削除する
#    sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", sentence)
  # 記号なども削除する
#    sentence = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", sentence)
  # スペース区切で形態素の配列に変換する
    wakati = sentence.split(" ")
  # 空要素を削除する
    wakati = list(filter(("").__ne__, wakati))
    return wakati

In [10]:
wakati_sentences = []

for i in range(len(sentences)):
    wakati_sentences.append(make_wakati(sentences[i]))

In [11]:
wcount = 256

emptylist = []
ssentences = np.append(emptylist, copy.deepcopy(sentences))

emplist = []
sectionlist = []

for i in enumerate(wakati_sentences):
    emp = 0
    section = 1
    if len(i[1])>wcount:
        wcount = 128
        count = 0
        countend = 0
        ssentences[i[0]] = []
        while len(i[1])-count-wcount>0:
            oneph = ''
            countend_ = 1
            while countend_%wcount != 0:
                oneph += i[1][countend]
                countend+=1
                countend_+=1
            ssentences[i[0]].append(oneph)
            count += wcount-1
            section += 1
        oneph = ''
        for j in range(len(i[1][count:-1])):
            oneph += i[1][count]
            count += 1
            emp += 1
        emplist.append(emp)
        ssentences[i[0]].append('')
        ssentences[i[0]][-1] = oneph
        sectionlist.append(section)
    else:
        oneph = ''
        for k in range(len(i[1])):
            oneph += i[1][k]
            ssentences[i[0]] = oneph
        emp = wcount - len(i[1])
        emplist.append(emp)
        sectionlist.append(1)

In [12]:
ssentences[0]

['既にさまざまなメディアで報じられているが東日本大震災後、結婚相談所を訪れる女性が急増したり、結婚指輪の売上が増加しているという。今まで結婚に興味がなかった独身者たちが転じて結婚に前向きな行動をとっているようだ。都内の企業に勤めるトモさん37歳は東日本大震災の時、帰宅困難になりましたよね。彼氏がいる同僚たちは彼氏と連絡を取り合って徒歩で帰りました。私は会社に泊まるか徒歩で帰るか悩んだあげく、一人では心細いので会社で',
 '一夜を明かしました。また、上司や先輩は家族の無事を確認していて。その時、誰かと一緒にいるって羨ましいなと心底思ったんですミナさん36歳派遣社員は派遣なので半年ごとに契約の更新があります。今回の震災でこれから財政が苦しくなりますよね。もちろん景気も悪化するでしょうし。いつ契約を切られるか正直不安です。結婚していればパートで働けるし、今より収入が減ったとしても旦那さんと二人で頑張れば心強いですよね。それが好き',
 'な人ならなおさら頑張れますし。あ結婚したい！って心から思ったの今回がはじめてですモモコさん35歳契約社員は度重なる余震と原発問題で日怯えています。ネットで地震予知をしている人のブログをついつい読んでしまって怖くなったり、節電なのにテレビをつけていないと心細いし、緊急地震速報のアラームが鳴る度に心臓がバクバクしてしまうし、一人の生活に耐えられないです。こんな時、好きな人や大切な人がそばに居てくれたらって思います。将来的に',
 '仕事もお金も不安ですが、なにより孤独ということ。このままおひとりさまの生涯を考えると不安で泣きたくなります今回、話をうかがった三人の共通点は、今までまったく結婚に興味がなかったということだ。一人の生活に慣れていたし、結婚している友人たちの話を聞いていても羨ましいと思うこともなかったと言う。むしろ、好きな時に好きなことをやって、自分のためにお金を使える今の暮らしに満足していた。しかし、今回の震災の',
 '影響もあってか、今後一人で生きていくというシビアな現実がリアルに目の前に飛び込んできたようだ。そう言えば先日、シングル女性だけでなくシングル男性たちも同じ思いをしているかもしれないと思うことがあった。友人男性から会社の後輩32歳が彼女を探しているんだけど、誰か独身の友達紹介してくれない？という連絡があった。友人

# BERT Tokenizerを用いて単語分割・IDへ変換
## Tokenizerの準備
単語分割とIDへ変換

# テスト実行

In [13]:
w_input_ids = []
w_attention_masks = []

for sent in ssentences:
    p_input_ids = []
    p_attention_masks = []
    for sect in sent:
        sencoded_dict = tokenizer.encode_plus(
                            sect,                      
                            add_special_tokens = True, # Special Tokenの追加
                            max_length = wcount+2,  # I think maximum 文章の長さを固定（Padding/Trancatinating）
                            truncation=True,                
                            pad_to_max_length = True,# PADDINGで埋める
                            return_attention_mask = True,   # Attention maksの作成
                            return_tensors = 'pt',     #  Pytorch tensorsで返す
                       )
        p_input_ids.append(torch.tensor(sencoded_dict['input_ids']).view(-1))
        p_attention_masks.append(torch.tensor(sencoded_dict['attention_mask']).view(-1))
    w_input_ids.append(p_input_ids)
    w_attention_masks.append(p_attention_masks)

In [14]:
# nagasa soroeru yo - id
pad = torch.full((1,130),3).view(-1)
maxlen = max(sectionlist)

for i in range(len(w_input_ids)):
    if maxlen>len(w_input_ids[i]):
        while maxlen>len(w_input_ids[i]):
            w_input_ids[i].append(pad)

In [15]:
# nagasa soroeru yo - attention
pad = torch.full((1,130),0).view(-1)

for i in range(len(w_attention_masks)):
    if maxlen>len(w_attention_masks[i]):
        while maxlen>len(w_attention_masks[i]):
            w_attention_masks[i].append(pad)

In [16]:
len(w_input_ids)
len(sectionlist)

1740

In [18]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import random_split, RandomSampler, SequentialSampler
import torchvision.transforms as transforms
from transformers import RobertaForMaskedLM
import random

# データローダーの作成
data_transform = transforms.Compose([
    synreplace(1),
    randinsert(3),
    randdelete(0.15), 
    randswap(2),
    transforms.ToTensor()
])

class MyDatasets(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, labels, valids, transform=None):
        self.ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels
        self.valids = valids
        self.transform = transform
        
    def __getitem__(self, idx):
        xa, mask, label, valid = self.ids[idx], self.attention_mask[idx], self.labels[idx], self.valids[idx]
        if self.transform:
            xa = self.transform(xa)
        return xa, mask, [label]*len(xa), valid

    def __len__(self):
        return len(self.ids)

    # 80%地点のIDを取得
num_dataset = len(w_input_ids)
train_size = 100
val_size = num_dataset - train_size

print('訓練データ数:{}'.format(train_size))
print('検証データ数:{}'.format(val_size))

# データセットクラスの作成 ichigyoume ha randamu shitei
indices = np.random.choice(num_dataset, num_dataset, replace=False)
wt_input_ids = [w_input_ids[i] for i in indices[:train_size]]
wt_attention_masks = [w_attention_masks[i] for i in indices[:train_size]]
wt_labels = [labels[i] for i in indices[:train_size]]
#wt_values = [value[i] for i in indices[:train_size]]
wt_values = [sectionlist[i] for i in indices[:train_size]]
wv_input_ids = [w_input_ids[i] for i in indices[train_size:]]
wv_attention_masks = [w_attention_masks[i] for i in indices[train_size:]]
wv_labels = [labels[i] for i in indices[train_size:]]
#wv_values = [value[i] for i in indices[train_size:]]
wv_values = [sectionlist[i] for i in indices[train_size:]]

train_dataset = MyDatasets(wt_input_ids, wt_attention_masks, wt_labels, wt_values)
val_dataset = MyDatasets(wv_input_ids, wv_attention_masks, wv_labels, wv_values)

# データローダーの作成
batch_size = 50

# 訓練データローダー
# shuffle True/False to compare or not
train_dataloader = DataLoader(
            train_dataset,
            batch_size = batch_size,
            shuffle = True
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            batch_size = val_size,
            shuffle = False
        )

訓練データ数:50
検証データ数:1690


In [19]:
a = wt_input_ids[0][0].detach().numpy()
# 50 kyoushi de-ta, saidai 46 block, 128+2
a[0]

9

In [22]:
from transformers import BertForSequenceClassification,AdamW,BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 2, # ラベル数（今回はBinaryなので2、数値を増やせばマルチラベルも対応可）
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
)

# モデルをGPUへ転送
model.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [23]:
# 最適化手法の設定
optimizer = AdamW(model.parameters(), lr=2e-5)

In [24]:
hoge = next(iter(train_dataloader))

In [25]:
# 学習の実行
max_epoch = 20
train_loss_ = []
test_loss_ = []

In [26]:
import scipy.stats as stats
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for ids, mask, labels, values in train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
#        b_input_ids = ids.to(device)
#        b_input_mask = mask.to(device)
#        b_labels = labels.to(device)
        b_input_ids = torch.stack(ids).to(device)
        b_input_mask = torch.stack(mask).to(device)
        b_labels = torch.stack(labels).to(device)
        optimizer.zero_grad()
        alloutputs = []
        for n in values:
            outputs = []
            for i in range(n):
                m_input_ids = b_input_ids[i]
                m_input_mask = b_input_mask[i]
                m_label = b_labels[i]
                output = model(m_input_ids, 
                             token_type_ids=None, 
                             attention_mask=m_input_mask, 
                             labels=m_label)
                loss = output.loss
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                train_loss += loss.item()
#                print(output)
#                print(output['logits'])
                outputs.append(output['logits'].to('cpu'))
            alloutputs.append(outputs)
    return train_loss, alloutputs

def validation(model):
    model.eval()# 訓練モードをオフ
    val_loss = 0
    alloutputs = []
    with torch.no_grad(): # 勾配を計算しない
        for ids, mask, labels, values in validation_dataloader:
    #        b_input_mask = mask.to(device)
    #        b_labels = labels.to(device)
            b_input_ids = torch.stack(ids).to(device)
            b_input_mask = torch.stack(mask).to(device)
            b_labels = torch.stack(labels).to(device)
            for n in values:
                outputs = []
                for i in range(n):
                    m_input_ids = b_input_ids[i]
                    m_input_mask = b_input_mask[i]
                    m_label = b_labels[i]
                    output = model(m_input_ids, 
                                 token_type_ids=None, 
                                 attention_mask=m_input_mask, 
                                 labels=m_label)
                    loss = output.loss
                    val_loss += loss.item()
                    outputs.append(output['logits'].to('cpu'))
                alloutputs.append(outputs)
    return val_loss, alloutputs

In [27]:
# nagasa soroeru yo
pad = torch.full((1,130),3).view(-1)
maxlen = max(sectionlist)

for i in range(len(w_input_ids)):
    if maxlen>len(w_input_ids[i]):
        while maxlen>len(w_input_ids[i]):
            w_input_ids[i].append(pad)

In [28]:
for epoch in range(max_epoch):
    train_ = train(model)
    train_loss_.append(train_)
    if epoch%10 == 0:
        print('epoch: ', epoch, 'train: ', train_)

epoch:  0 train:  (271.58099011634476, [[tensor([[-0.2715,  1.8781],
        [-0.1366,  1.8663]], grad_fn=<ToCopyBackward0>), tensor([[-0.0984,  1.5315],
        [ 0.0663,  1.9089]], grad_fn=<ToCopyBackward0>), tensor([[-0.3785,  1.5640],
        [-0.2498,  1.4280]], grad_fn=<ToCopyBackward0>), tensor([[-0.2084,  1.4365],
        [-0.2201,  1.5153]], grad_fn=<ToCopyBackward0>)], [tensor([[ 0.0271,  1.4500],
        [-0.0862,  1.4417]], grad_fn=<ToCopyBackward0>), tensor([[-0.0027,  1.5225],
        [-0.2749,  1.2250]], grad_fn=<ToCopyBackward0>), tensor([[-0.1954,  1.2623],
        [-0.2150,  1.3923]], grad_fn=<ToCopyBackward0>), tensor([[-0.1900,  1.3703],
        [ 0.0239,  1.1830]], grad_fn=<ToCopyBackward0>), tensor([[-1.7970,  2.0838],
        [-0.2041,  1.5080]], grad_fn=<ToCopyBackward0>), tensor([[-1.7214,  2.0241],
        [-0.1093,  1.3362]], grad_fn=<ToCopyBackward0>)]])


In [29]:
test_loss_ = validation(model)
print('test: ', test_loss_)

test:  (7376.107897341251, [[tensor([[0.7746, 0.5844],
        [0.7747, 0.5838],
        [0.7747, 0.5839],
        ...,
        [0.7747, 0.5840],
        [0.7747, 0.5840],
        [0.7748, 0.5838]]), tensor([[0.7747, 0.5843],
        [0.7747, 0.5839],
        [0.7748, 0.5838],
        ...,
        [0.7748, 0.5838],
        [0.7747, 0.5839],
        [0.7747, 0.5839]]), tensor([[0.7747, 0.5841],
        [0.7747, 0.5839],
        [0.7747, 0.5839],
        ...,
        [0.7747, 0.5838],
        [0.7747, 0.5845],
        [0.7747, 0.5839]]), tensor([[0.7746, 0.5844],
        [0.7747, 0.5841],
        [0.7747, 0.5839],
        ...,
        [0.7748, 0.5838],
        [0.7748, 0.5841],
        [0.7747, 0.5839]]), tensor([[0.7748, 0.5843],
        [0.7748, 0.5841],
        [0.7748, 0.5838],
        ...,
        [0.7747, 0.5839],
        [0.7748, 0.5841],
        [0.7748, 0.5838]])], [tensor([[0.7746, 0.5844],
        [0.7747, 0.5838],
        [0.7747, 0.5839],
        ...,
        [0.7747, 0.5840

In [30]:
# b_input_mask.size(), b_input_ids.size(), labels.size()
# outputs = self.model(torch.unsqueeze(token_tensor, 0))

In [31]:
test_loss_[0]# all loss
test_loss_[1] # 1690
test_loss_[1][0] # burokkusuu
test_loss_[1][0][0] # batch ikko niha shita
test_loss_[1][0][0][0] # hoshii yatsu

tensor([0.7746, 0.5844])

In [32]:
test_loss_[1][0][4][0]

tensor([0.7748, 0.5843])

In [33]:
len(wv_labels)

1690

# HOUHOU 1

In [34]:
methodone = []
for i in range(len(test_loss_[1])):
    article = []
    for j in range(len(test_loss_[1][i])):
        block = np.argmax(test_loss_[1][i][j][0].numpy())
        article.append(block)
    articlesum = np.sum(np.array(article))
    if articlesum/len(test_loss_[1][i]) <= 0.5:
        methodone.append(0)
    else:
        methodone.append(1)

# HOUHOU2

In [35]:
methodtwo = []
for i in range(len(test_loss_[1])):
    article = [0,0]
    for j in range(len(test_loss_[1][i])):
        block = test_loss_[1][i][j][0].numpy()
        article = [x+y for (x,y) in zip(article, block)]
    articlesum = np.argmax(np.array(article))
    if articlesum <= 0.5:
        methodtwo.append(0)
    else:
        methodtwo.append(1)

In [36]:
# nanka houhou 2 ga umaku ittenai kamo
# seikai tono hikaku shitai ne
len(methodtwo)

1690

In [43]:
one_df = pd.DataFrame(methodone, columns=['method1'])
two_df = pd.DataFrame(methodtwo, columns=['method2'])
label_df = pd.DataFrame(wv_labels, columns=['true_label'])
accuracy_df = pd.concat([one_df, two_df, label_df], axis=1)
accuracy_df.head(50)

,method1,method2,true_label
0,0,0,1
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,1
5,0,0,1
6,0,0,1
7,0,0,1
8,0,0,1
9,0,0,0


In [38]:
def accuracy(pdf):
    cor = 0
    for i in range(len(wv_labels)):
        if pdf[i] == label_df.values[i]:
            cor += 1
    acc = 100*cor/len(pdf)
    return acc

def fscore(pdf):
    cor = 0
    ypnum = 0 #yosoku
    spnum = 0 #seikai
    pnum = 0
    rnum = 0
    fone = 0
    for i in range(len(wv_labels)):
        if pdf[i] == label_df.values[i]:
            cor += 1
        if pdf[i] == 0:
            ypnum += 1
            if label_df.values[i] == 0:
                pnum += 1
        if label_df.values[i] == 0:
            spnum += 1
            if pdf[i] == 0:
                rnum += 1
    # tekigou
    tp = pnum/ypnum
    # saigen
    sp = rnum/spnum
    # f1
    fone = (tp*sp)/(tp+sp)
    return fone

In [39]:
onepreds = one_df.values
twopreds = two_df.values

print('methodone: ', accuracy(onepreds), fscore(onepreds))
print('methodtwo: ', accuracy(twopreds), fscore(twopreds))

methodone:  50.17751479289941 0.3341213553979511
methodtwo:  50.17751479289941 0.3341213553979511
